In [1]:
from dataset_collection import dataset
from utils import get_dataset, get_model
import cv2
font = cv2.FONT_HERSHEY_DUPLEX
import numpy as np 
import keras
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils.np_utils import to_categorical   

In [3]:
## provide line and not-live data
dataset()

In [4]:
## Getting the dataset

live_img, live_img_label, not_live, not_live_label = get_dataset()

img = live_img + not_live
labels = live_img_label+ not_live_label
images=[]
for i in img:
    img = cv2.imread(i)
    img = cv2.resize(img, (100,100))
    images.append(img)

X = np.array(images, dtype=float)
y = np.array(labels, dtype=float)
y= y.reshape((-1,1))
X /= 255
y = to_categorical(y, num_classes=2)

In [5]:
X.shape, y.shape

((2000, 100, 100, 3), (2000, 2))

In [6]:
## Model Training

model = get_model()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit(x=X, y=y, validation_split=0.3, epochs=3)

Epoch 1/3
44/44 [==============================] - 41s 911ms/step - loss: 0.3408 - acc: 0.8110 - val_loss: 0.0148 - val_acc: 1.0000
Epoch 2/3
44/44 [==============================] - 39s 881ms/step - loss: 0.0069 - acc: 1.0000 - val_loss: 4.5961e-04 - val_acc: 1.0000
Epoch 3/3
44/44 [==============================] - 38s 876ms/step - loss: 4.0225e-04 - acc: 1.0000 - val_loss: 0.1266 - val_acc: 0.9650


In [7]:
## Save weights
# model.save_weights('weights.h5')

model = get_model()
model.load_weights('weights.h5')

In [8]:
########## KNN CODE ############
def distance(v1, v2):
    return np.sqrt(((v1-v2)**2).sum()) ## Eucledian

def knn(train, test, k=5):
    dist = []

    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
        
    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]

    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]
################################

In [9]:
dataset_path = "face_dataset/"

face_data = []
labels = []
class_id = 0
names = {}

In [10]:
import os

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)

        target = class_id * np.ones((data_item.shape[0],))    ##names= {[0,vasu],[1,vinayak],....so on}
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1, 1))
print(face_labels.shape)
print(face_dataset.shape)

trainset = np.concatenate((face_dataset, face_labels), axis=1)
print(trainset.shape)

(103, 1)
(103, 30000)
(103, 30001)


In [15]:
cap = cv2.VideoCapture(0)

result = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, (640, 480))

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    img = cv2.resize(frame, (100,100))
    img = np.expand_dims(img, axis=0)
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    pred = model.predict(img)
    
    if pred[0][1]:
        cv2.putText(frame, '!!WARNING!!', (frame.shape[1]//2, frame.shape[0]//2), font, 1.0, (255, 255, 255), 1)        
    
    else:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:

            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            
            offset = 5
            face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
            face_section = cv2.resize(face_section, (100, 100))
            
            out = knn(trainset, face_section.flatten())
            cv2.putText(frame, names[int(out)],(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),2,cv2.LINE_AA)
        
        cv2.putText(frame, 'UNLOCKED', (frame.shape[1]//2, frame.shape[0]//2), font, 1.0, (0, 255, 0), 1)

    result.write(frame)

    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
result.release()
cv2.destroyAllWindows()